In [78]:
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
import scanpy as sc
import numpy as np

# Data

In [91]:
df_allfiles = pd.read_csv("../miRNA/files.dat", index_col=0)
df_allfiles["level_0"][:2], df_allfiles["index"][:2]

(cases.0.submitter_id
 TCGA-D8-A140    67414fe8-5dfa-49e0-a3a9-a696a15be65f.mirbase21...
 TCGA-BH-A0BA    97fb1d84-7fa0-4fa9-b5ed-c370cb774c6f.mirbase21...
 Name: level_0, dtype: object,
 cases.0.submitter_id
 TCGA-D8-A140    1c072d05-bdd8-4d30-ba1d-099ed32ea33a.FPKM.txt.gz
 TCGA-BH-A0BA    5c0f9cb8-cc53-49ce-8384-6acb54cb1855.FPKM.txt.gz
 Name: index, dtype: object)

# Process miRNA

In [84]:
df_mirna = pd.read_csv("../miRNA/mainTable_miRNA.csv", index_col=0).reindex(columns=df_allfiles["level_0"].values)

In [85]:
X = df_mirna.transpose().values

In [86]:
lda = LatentDirichletAllocation(n_jobs=12)
topics = lda.fit_transform(X)

In [87]:
topics.shape

(780, 10)

In [88]:
np.savetxt("train-label.txt", topics.argmax(1).T, fmt="%d")

## Save data

In [97]:
df =  pd.read_csv("../miRNA/mainTable_fpkm.csv", index_col=0).reindex(columns=df_allfiles["index"])
df_files= pd.read_csv("../miRNA/files_fpkm.dat", index_col=0).reindex(index=df.columns)

In [98]:
adata = sc.AnnData(X = df.transpose(), obs=df_files)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=3000)

In [99]:
df = df.reindex(index=adata.var[adata.var["highly_variable"]].index).applymap(lambda fpkm: np.log2(fpkm+1))
df.head(2)

index,1c072d05-bdd8-4d30-ba1d-099ed32ea33a.FPKM.txt.gz,5c0f9cb8-cc53-49ce-8384-6acb54cb1855.FPKM.txt.gz,445b096b-502f-4e88-9338-4ad309a14425.FPKM.txt.gz,a0c5137f-e9b4-413c-a0b3-96c2c11e68f2.FPKM.txt.gz,1f4c4871-667d-4310-a45a-4a5d5f6b60eb.FPKM.txt.gz,762e3c02-fbce-4778-8d79-8fa732fd399e.FPKM.txt.gz,ef647ec0-3322-4dd2-84ca-83bcd786b765.FPKM.txt.gz,5029a88f-ac46-4a7e-afd0-398cfb17bd5f.FPKM.txt.gz,46ee3218-2a1f-4d44-8706-d44d7e4b980d.FPKM.txt.gz,07ad35fc-1e35-43b3-a252-193323d5e680.FPKM.txt.gz,...,0f8ed930-950b-4bae-a02d-e598b74bd82b.FPKM.txt.gz,9ce4048c-b607-4677-b874-647e9c29785d.FPKM.txt.gz,96833a09-14ce-4483-b17d-f4328e0b68f5.FPKM.txt.gz,cc55afdc-78e2-48c5-b1ac-386033852bca.FPKM.txt.gz,f9365670-8213-4263-80aa-1718e6d05dc8.FPKM.txt.gz,21b33b4b-04e9-4205-919f-8ec6b2e0cca0.FPKM.txt.gz,bc4753f9-5ca7-4bc8-b25a-622e1a4cdf5d.FPKM.txt.gz,27a9c6bd-6e26-4055-867c-ab7276492464.FPKM.txt.gz,aa256c1a-55a8-43b1-a553-26444a950087.FPKM.txt.gz,b4824c2d-fed2-4fad-bf6f-797119546006.FPKM.txt.gz
0,,,,,,,,,,,,,,,,,,,,,
ENSG00000000938,1.405992,1.505891,2.695994,1.459432,1.669027,2.070389,2.801159,3.152183,2.035624,3.030336,...,1.137504,2.077243,1.195348,1.863938,1.627607,1.819668,1.565597,0.807355,1.056584,0.454176
ENSG00000000971,2.615887,2.883621,2.996389,2.601697,2.384050,3.152183,2.950468,1.097611,2.771886,3.150560,...,2.414136,2.304511,2.505891,3.476382,2.577731,2.160275,2.353323,1.070389,2.070389,0.536053


In [100]:
with open("train-data.txt","w") as file:
    for sample in df.columns:
        doc = df[sample].round().astype(int)
        file.write(f"{len(doc[doc>0])} ")
        for el in [f"{iw}:{w[1]}" for iw,w in enumerate(doc.items()) if w[1] > 0]:
            file.write(el+" ")
        file.write("\n")

# Postprocess

In [109]:
df_topics = pd.read_csv("mirna/final.gamma", sep=" ", header=None)
df_topics.index = df_allfiles.index
df_topics.columns = ["Topic %d"%(t+1) for t in range(df_topics.shape[1])]
df_topics.head(2)

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10
cases.0.submitter_id,,,,,,,,,,
TCGA-D8-A140,626.882681,634.896929,644.657135,659.632651,647.695838,637.938957,645.040123,635.913303,646.166438,640.175944
TCGA-BH-A0BA,684.965520,673.164344,675.130579,670.772948,673.195540,689.713523,696.454567,695.427102,674.237686,689.938192


In [111]:
df.head(1)

index,1c072d05-bdd8-4d30-ba1d-099ed32ea33a.FPKM.txt.gz,5c0f9cb8-cc53-49ce-8384-6acb54cb1855.FPKM.txt.gz,445b096b-502f-4e88-9338-4ad309a14425.FPKM.txt.gz,a0c5137f-e9b4-413c-a0b3-96c2c11e68f2.FPKM.txt.gz,1f4c4871-667d-4310-a45a-4a5d5f6b60eb.FPKM.txt.gz,762e3c02-fbce-4778-8d79-8fa732fd399e.FPKM.txt.gz,ef647ec0-3322-4dd2-84ca-83bcd786b765.FPKM.txt.gz,5029a88f-ac46-4a7e-afd0-398cfb17bd5f.FPKM.txt.gz,46ee3218-2a1f-4d44-8706-d44d7e4b980d.FPKM.txt.gz,07ad35fc-1e35-43b3-a252-193323d5e680.FPKM.txt.gz,...,0f8ed930-950b-4bae-a02d-e598b74bd82b.FPKM.txt.gz,9ce4048c-b607-4677-b874-647e9c29785d.FPKM.txt.gz,96833a09-14ce-4483-b17d-f4328e0b68f5.FPKM.txt.gz,cc55afdc-78e2-48c5-b1ac-386033852bca.FPKM.txt.gz,f9365670-8213-4263-80aa-1718e6d05dc8.FPKM.txt.gz,21b33b4b-04e9-4205-919f-8ec6b2e0cca0.FPKM.txt.gz,bc4753f9-5ca7-4bc8-b25a-622e1a4cdf5d.FPKM.txt.gz,27a9c6bd-6e26-4055-867c-ab7276492464.FPKM.txt.gz,aa256c1a-55a8-43b1-a553-26444a950087.FPKM.txt.gz,b4824c2d-fed2-4fad-bf6f-797119546006.FPKM.txt.gz
0,,,,,,,,,,,,,,,,,,,,,
ENSG00000000938,1.405992,1.505891,2.695994,1.459432,1.669027,2.070389,2.801159,3.152183,2.035624,3.030336,...,1.137504,2.077243,1.195348,1.863938,1.627607,1.819668,1.565597,0.807355,1.056584,0.454176


In [110]:
pd.read_csv("mirna/word-assignments.dat")

2286 0000:07 0001:05 0002:04 0003:01 0004:06 0005:09 0006:02 0007:07 0008:05 0009:02 0010:09 0011:07 0012:09 0013:05 0014:00 0016:09 0017:04 0018:01 0019:09 0021:06 0022:09 0023:07 0024:06 0026:06 0027:05 0028:03 0029:02 0030:05 0032:06 0033:08 0034:05 0036:00 0037:00 0039:03 0040:05 0042:02 0043:06 0044:06 0045:03 0046:08 0047:01 0048:05 0049:07 0050:01 0051:04 0052:00 0053:09 0054:00 0055:04 0056:01 0059:04 0060:04 0061:04 0062:03 0063:07 0065:02 0066:05 0067:02 0068:07 0069:01 0070:04 0071:09 0072:00 0073:00 0075:08 0076:01 0077:03 0078:03 0079:03 0082:04 0083:02 0084:00 0085:00 0086:03 0088:07 0089:03 0090:08 0091:08 0092:08 0093:04 0094:04 0095:05 0096:09 0097:05 0098:02 0099:00 0100:00 0101:01 0102:08 0103:03 0104:04 0105:05 0107:07 0108:00 0109:02 0110:03 0111:03 0112:09 0113:07 0114:04 0115:05 0116:03 0117:05 0118:07 0119:09 0120:06 0121:08 0122:05 0124:08 0125:00 0126:06 0127:04 0128:06 0129:08 0130:06 0131:03 0132:03 0133:01 0134:06 0135:08 0136:02 0137:04 0138:08 0139:04 0140:08 0141:04 0143:03 0144:05 0145:00 0146:02 0147:09 0148:06 0149:02 0150:06 0151:05 0152:06 0153:05 0154:01 0155:09 0156:03 0157:02 0158:09 0159:07 0160:01 0161:04 0162:05 0163:08 0164:07 0165:09 0166:07 0167:06 0168:04 0169:07 0171:01 0172:02 0173:02 0175:04 0176:06 0177:04 0178:00 0179:08 0180:03 0181:03 0182:07 0183:09 0184:05 0185:07 0186:09 0187:01 0188:08 0190:04 0191:02 0192:09 0194:00 0195:05 0196:09 0197:05 0198:03 0199:04 0200:08 0201:01 0202:09 0203:01 0204:04 0205:08 0206:07 0208:05 0210:04 0211:03 0212:01 0214:02 0215:00 0216:01 0217:01 0218:05 0220:02 0221:00 0222:08 0223:07 0224:09 0225:03 0226:08 0227:02 0229:04 0230:05 0231:08 0232:08 0233:03 0235:05 0236:05 0238:07 0239:04 0240:00 0241:03 0242:05 0243:03 0244:04 0246:06 0248:02 0249:09 0250:07 0251:06 0253:04 0254:07 0255:07 0256:04 0257:03 0258:04 0259:03 0260:05 0261:01 0263:05 0264:05 0268:05 0269:02 0271:03 0272:06 0274:03 0275:05 0276:05 0278:05 0279:03 0280:08 0281:03 0282:05 0283:04 0284:04 0285:09 0286:04 0287:03 0289:03 0290:08 0291:01 0292:07 0293:06 0294:06 0295:06 0296:03 0297:02 0298:02 0299:04 0300:01 0301:09 0302:03 0303:04 0304:03 0305:05 0306:01 0307:04 0308:03 0309:00 0311:02 0312:06 0314:06 0315:07 0317:06 0318:04 0319:08 0320:05 0321:08 0322:07 0323:03 0324:04 0325:03 0326:04 0328:03 0330:07 0331:03 0332:04 0333:04 0334:03 0335:02 0337:04 0339:02 0340:03 0341:02 0342:04 0343:07 0344:03 0345:09 0346:08 0347:06 0348:04 0349:07 0350:09 0351:02 0352:06 0353:06 0354:01 0355:05 0356:04 0358:07 0359:03 0360:06 0361:07 0362:04 0364:08 0365:07 0366:03 0367:06 0368:07 0369:07 0370:03 0371:09 0372:01 0373:06 0374:04 0375:09 0376:05 0377:06 0379:08 0380:01 0381:03 0382:03 0383:07 0384:08 0386:01 0387:04 0388:04 0389:08 0390:03 0391:05 0392:07 0393:02 0394:05 0395:09 0396:05 0397:02 0398:02 0399:04 0400:05 0402:09 0403:03 0404:06 0405:05 0406:06 0408:04 0410:07 0411:07 0412:05 0413:03 0414:00 0415:05 0416:08 0417:08 0418:04 0420:06 0421:02 0422:06 0423:06 0424:08 0425:05 0426:08 0427:06 0429:02 0430:05 0431:04 0432:01 0433:09 0434:03 0436:07 0437:01 0438:01 0440:07 0441:04 0442:00 0443:08 0445:01 0446:02 0447:04 0448:02 0452:08 0453:04 0454:08 0455:03 0456:04 0457:09 0458:04 0462:08 0463:07 0464:07 0465:09 0466:04 0468:04 0469:09 0470:03 0471:01 0472:02 0473:07 0474:06 0475:03 0476:06 0477:06 0478:08 0479:03 0481:09 0482:07 0483:07 0484:07 0485:06 0486:05 0489:04 0490:04 0491:00 0492:08 0493:06 0494:07 0495:01 0496:06 0497:00 0499:04 0502:02 0503:02 0504:02 0506:04 0507:06 0508:07 0509:05 0511:03 0513:05 0514:08 0515:08 0516:07 0517:07 0518:07 0519:05 0520:04 0521:06 0522:02 0523:03 0524:04 0525:04 0526:07 0527:03 0528:07 0529:09 0530:09 0531:03 0533:09 0534:02 0535:04 0536:09 0537:03 0538:06 0539:07 0540:01 0541:01 0542:09 0543:07 0544:09 0545:04 0548:09 0549:00 0550:08 0551:05 0552:05 0553:08 0554:09 0555:08 0556:06 0557:02 0558:08 0560:01 0561:06 0563:05 0564:04 0565:04 0566:01 0567:07 0568:03 0569:01 0570:03 0571:03 0572:00 0573:00 0574:09 0575:03 0576:03 0577:06 057